In [ ]:
"""
@author: Alain Plantec

Voici un skelette possible.
Vous devez programmez les classes contenues dans ce squelette.
Les fonctions et leurs paramètres ainsi que les variables contenues dans le classes
ont du sens par rapport à une version programmée pour la préparation du projet.
Votre version sera forcément différente.
Donc, vous pouvez ajouter/retirer des variables et/ou des fonctions et/ou des paramètres.

"""
try:  # import as appropriate for 2.x vs. 3.x
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox

from sokobanXSBLevels import *
from enum import Enum
import time
"""
Direction :
    Utile pour gérer le calcul des positions pour les mouvements
"""
class Direction(Enum):
    Up = 1
    Down = 2
    Left = 3
    Right = 4

"""
Position :
    - stockage de coordoannées x et y,
    - vérification de x et y par rapport à une matrice
    - calcule de position relative à partir d'un offset (un décalage) et une direction 
"""
class Position(object):
    def __init__(self, x, y):
        # Initialisation d'une position avec des coordonnées x et y.
        self.x = x
        self.y = y

    def __str__(self):
        # Représentation sous forme de chaîne de caractères de la position.
        # Cela permet de facilement afficher un objet Position, par exemple dans un print.
        return 'Position(' + str(self.x) + ',' + str(self.y) + str(')') 

    def getX(self):
        # Retourne la coordonnée X de la position.
        return self.x

    def getY(self):
        # Retourne la coordonnée Y de la position.
        return self.y

    def positionTowards(self, direction, offset):
        # Cette méthode retourne une nouvelle position à partir de la position actuelle,
        # en fonction de la direction donnée (par exemple, gauche, droite, haut, bas) et d'un offset (déplacement).
        # Exemple : Position(3,4).positionTowards(Direction.Right, 2) retourne Position(5,4).
        if direction == Direction.Left:
            return Position(self.x - offset, self.y)  # Déplace la position vers la gauche.
        elif direction == Direction.Up:
            return Position(self.x , self.y - offset)  # Déplace la position vers le haut.
        elif direction == Direction.Right:
            return Position(self.x + offset, self.y)  # Déplace la position vers la droite.
        elif direction == Direction.Down:
            return Position(self.x, self.y + offset)  # Déplace la position vers le bas.

    def isValidInWharehouse(self, wharehouse):
        # Vérifie si la position est valide dans l'entrepôt (wharehouse).
        # Cela se fait en vérifiant si la position donnée existe dans la matrice de l'entrepôt.
        return wharehouse.isPositionValid(self)

    def asCanvasPositionIn(self, elem):
        # Cette méthode convertit la position en coordonnées adaptées pour un canvas graphique.
        # Elle prend en compte la largeur et la hauteur de l'élément pour calculer la position correcte sur le canvas.
        lx = self.getX() * elem.getWidth()  # Calcule la coordonnée X sur le canvas.
        ly = self.getY() * elem.getHeight()  # Calcule la coordonnée Y sur le canvas.
        return Position(lx, ly)  # Retourne la position calculée sur le canvas.

"""
WharehousePlan : Plan de l'entrepot pour stocker les éléments.
    Les éléments sont stockés dans une matrice (#rawMatrix)
"""
class WharehousePlan(object):
    def __init__(self):
        # Initialisation de la matrice brute (rawMatrix) qui contiendra les éléments de l'entrepôt.
        # Cette matrice est une liste de listes représentant l'entrepôt.
        self.rawMatrix = []

    def appendRow(self, row):
        # Ajoute une nouvelle ligne à la matrice rawMatrix.
        # Cette ligne représente un ensemble d'éléments (par exemple, des cases dans l'entrepôt).
        self.rawMatrix.append(row) 

    def at(self, position):
        # Cette méthode retourne l'élément situé à la position donnée (x, y).
        # Elle récupère l'élément dans la matrice à l'index correspondant aux coordonnées de la position.
        return self.rawMatrix[position.getY()][position.getX()] 

    def atPut(self, position, elem):
        # Cette méthode met à jour la matrice rawMatrix en plaçant un nouvel élément (elem)
        # à la position spécifiée (x, y). Elle vérifie d'abord que la position est valide.
        self.rawMatrix[position.getY()][position.getX()] = elem 

    def isPositionValid(self, position):
        # Cette méthode vérifie si la position donnée est valide dans la matrice.
        # Elle essaie de récupérer l'élément à la position donnée via la méthode `at`.
        # Si un IndexError est levé, cela signifie que la position est hors de la matrice, donc invalide.
        try:
            self.at(position)
        except IndexError:
            return False  # La position est invalide.
        else:
            return True  # La position est valide.

    def hasFreePlaceAt(self, position):
        # Cette méthode vérifie si l'élément à la position donnée est une place libre.
        # Elle appelle la méthode `isFreePlace` de l'élément à la position donnée pour savoir s'il y a une place libre.
        return self.at(position).isFreePlace() 

    def asXsbMatrix(self):
        # Cette méthode convertit la matrice `rawMatrix` en un format spécifique au jeu (XsbMatrix).
        # La méthode retourne l'objet XsbMatrix qui représente l'entrepôt sous forme d'un format de niveau Sokoban.
        return xsbMatrix(self.rawMatrix)
"""
Floor :
    Représente une case vide de la matrice
    (pas de None dans la matrice)
"""
class Floor(object):
    def __init__(self):
        None
    def isMovable(self):
        return False
    def canBeCovered(self):
        return True
    def xsbChar(self):
        return ' '
    def isFreePlace(self):
        return True
"""
Goal :
    Représente une localisation à recouvrir d'un BOX (objectif du jeu).
    Le déménageur doit parvenir à couvrir toutes ces cellules à partir des caisses.
    Un Goal est static, il est toujours déssiné en dessous :
        Le zOrder est assuré par le tag du create_image (tag='static')
        et self.canvas.tag_raise("movable","static") dans Level
"""
class Goal(object):
    def __init__(self, canvas, position):
        self.canvas = canvas
        self.position = position 
        self.height = 64
        self.width = 64
        self.image = tk.PhotoImage(file='goal.png')
        self.canvas.create_image(self.position.getX()*64,self.position.getY()*64,image=self.image,anchor="nw", tags='static')

    def isMovable(self):
        return False

    def getHeight(self):
        return self.height
    
    def getWidth(self):
        return self.width

    def canBeCovered(self):
        return True
        
    def xsbChar(self):
        return '.'

    def isFreePlace(self):
        return False

"""
Wall : pour délimiter les murs
    Le déménageur ne peut pas traverser un mur.
    Un Wall est static, il est toujours déssiné en dessous :
        Le zOrder est assuré par le tag du create_image (tag='static')
        et self.canvas.tag_raise("movable","static") dans Level
"""
class Wall(object):
    def __init__(self, canvas, position):
        self.canvas = canvas
        self.position = position 
        self.height = 64
        self.width = 64
        self.image = tk.PhotoImage(file='wall.png')
        self.canvas.create_image(self.position.getX()*64,self.position.getY()*64,image=self.image,anchor="nw", tags='static')

    def getHeight(self):
        return self.height
    
    def getWidth(self):
        return self.width

    def isMovable(self):
        return False

    def canBeCovered(self):
        return False

    def xsbChar(self):
        return '#'

    def isFreePlace(self):
        return False

"""
Box : Caisse à déplacer par le déménageur.
    Etant donné qu'une caisse doit être déplacé, le canvas et la matrice sont necessaires pour
    reconstruire l'image et mettre en oeuvre sont déplacement (dans le canvas et dans la matrice)
    Un Box est "movable", il est toujours déssiné au dessus des objets "static" :
        Le zOrder est assuré par le tag du create_image (tag='movable')
        et self.canvas.tag_raise("movable","static") dans Level
    Un Box est représenté differemment (image différente) suivant qu'il se situe sur un emplacement marqué par un Goal ou non.
 """
class Box(object):
    def __init__(self, canvas, wharehouse, position, onGoal):
        # Le constructeur de la classe Box initialise un objet représentant une boîte dans le jeu Sokoban.
        # Il prend plusieurs paramètres pour définir l'état et la position de la boîte.

        self.canvas = canvas  # Référence au canevas tkinter où la boîte sera dessinée.
        self.position = position  # Position de la boîte dans l'entrepôt.
        self.wharehouse = wharehouse  # Référence à l'entrepôt (WharehousePlan) où la boîte se trouve.
        self.onGoal = onGoal  # Indicateur si la boîte est sur un objectif (goal).
        self.under = Floor()  # La case sous la boîte, initialisée à un sol vide (Floor).
        self.height = 64  # Hauteur de la boîte, en pixels.
        self.width = 64  # Largeur de la boîte, en pixels.

        # Vérifie si la boîte est sur un objectif (goal) au moment de la création et affiche l'image correspondante.
        # Si `onGoal` est vrai, l'image affichera la boîte sur un objectif, sinon la boîte normale.
        if onGoal:
            self.image = tk.PhotoImage(file='boxOnTarget.png')  # Image de la boîte sur un objectif.
        else:
            self.image = tk.PhotoImage(file='box.png')  # Image de la boîte normale.

        # Création de l'image de la boîte sur le canevas. Les coordonnées sont multipliées par 64
        # pour les adapter à l'échelle de la grille de jeu .
        self.box_id = self.canvas.create_image(self.position.getX() * 64, self.position.getY() * 64,image=self.image, anchor="nw", tags='movable')
    def getHeight(self):
        return self.height
    
    def getWidth(self):
        return self.width

    def isMovable(self):
        return True

    def canBeCovered(self):
        return False
     # cette methode sert à bouger le box
    def moveTowards(self, direction): 
             # il y aura ci dessous l'application de la meme logique sur tous les types de direction 
            if direction == Direction.Left:
                # Ici, le code vérifie si la case à la gauche de la boîte peut être recouverte. 
                # Si c'est le cas, le mouvement de la boîte est autorisé
                if self.wharehouse.at(self.position.positionTowards(direction,1)).canBeCovered():
                    # Cette ligne déplace visuellement la boîte vers la gauche dans le canvas. 
                    # La valeur -64 indique un déplacement de 64 pixels vers la gauche 
                    self.canvas.move(self.box_id,-64,0)
                    # La position de la boîte est mise à jour en conséquence du mouvement vers la gauche
                    self.position = self.position.positionTowards(direction,1)
                    # Cette ligne vérifie si la case suivante (après le déplacement) est un goal pour mettre a jour l'image de box 
                    if self.wharehouse.at(self.position.positionTowards(direction,0)).xsbChar() == '.':
                        self.startGoalCoveredAnimation()
                    # si le joueur déplace un box et le box n'est plus sur un goal, 
                    # il faut que son image change et retourne comme un box normal
                    else:
                        self.cleanUpAnimation()
                    
            elif direction == Direction.Up:
                 if self.wharehouse.at(self.position.positionTowards(direction,1)).canBeCovered():
                    self.canvas.move(self.box_id,0,-64)
                    self.position = self.position.positionTowards(direction,1)
                    if self.wharehouse.at(self.position.positionTowards(direction,0)).xsbChar() == '.':
                        self.startGoalCoveredAnimation()
                    else:
                        self.cleanUpAnimation()

            elif direction == Direction.Right:
                 if self.wharehouse.at(self.position.positionTowards(direction,1)).canBeCovered():
                    self.canvas.move(self.box_id,64,0)
                    self.position = self.position.positionTowards(direction,1)
                    if self.wharehouse.at(self.position.positionTowards(direction,0)).xsbChar() == '.':
                        self.startGoalCoveredAnimation()
                    else:
                        self.cleanUpAnimation()

            elif direction == Direction.Down:
                 if self.wharehouse.at(self.position.positionTowards(direction,1)).canBeCovered():
                    self.canvas.move(self.box_id,0,64)
                    self.position = self.position.positionTowards(direction,1)
                    if self.wharehouse.at(self.position.positionTowards(direction,0)).xsbChar() == '.':
                        self.startGoalCoveredAnimation()
                    else:
                        self.cleanUpAnimation()

    def xsbChar(self):
        if self.under.isFreePlace(): 
            return '$'
        else: 
            return '*'

    def isFreePlace(self):
        return False
    # une animation basée sur le changement de l'image de box lorqu'il arrive sur un goal 
    def startGoalCoveredAnimation(self):
        self.canvas.delete(self.box_id)
        self.image = tk.PhotoImage(file='boxOnTarget.png')
        self.box_id =self.canvas.create_image(self.position.getX()*64,self.position.getY()*64,image=self.image,anchor="nw", tags='movable')
     # une animation basée sur le changement de l'image de box lorqu'il quitte un goal
    def cleanUpAnimation(self):
        self.canvas.delete(self.box_id)
        self.image = tk.PhotoImage(file='box.png')
        self.box_id =self.canvas.create_image(self.position.getX()*64,self.position.getY()*64,image=self.image,anchor="nw", tags='movable')
 
    def goalCoveredAnimation(self):
         pass

"""
Mover : C'est  le déménageur.
    La classe Mover met en oeuvre la logique du jeu dans #canMove et #moveTowards.
    Etant donné qu'un Mover se déplace, le canvas et la matrice sont necessaires pour
    reconstruire l'image et mettre en oeuvre sont déplacement (dans le canvas et dans la matrice)
    Un Mover est "movable", il est toujours déssiné au dessus des objets "static" :
        Le zOrder est assuré par le tag du create_image (tag='movable')
        et self.canvas.tag_raise("movable","static") dans Level
    Un Box est représenté differemment (image différente) suivant la direction de déplacement (même si le dépplacement s'avère impossible).
"""

class Mover(object):
    def __init__(self, canvas, wharehouse, position, onGoal):
        # Mover initialise un objet représentant le joueur .
        # Il prend plusieurs paramètres pour définir l'état et la position du joueur.

        self.canvas = canvas  
        self.position = position  # Position actuelle du joueur dans l'entrepôt.
        self.height = 64  # Hauteur du joueur, en pixels (la taille d'une case).
        self.width = 64  # Largeur du joueur, en pixels (la taille d'une case).
        self.image = tk.PhotoImage(file='player.png') 
        self.player_id = self.canvas.create_image(
            self.position.getX() * 64, 
            self.position.getY() * 64,
            image=self.image, 
            anchor="nw", 
            tags='movable'
        )
        self.wharehouse = wharehouse  # l'entrepôt où le joueur se déplace.
        self.onGoal = onGoal  # si le joueur est sur un (goal).
        self.under = Floor()  # La case sous le joueur (initialisée à un sol vide).
        self.impossible_animation_running = False  # Empêche plusieurs animations en même temps

    def getHeight(self):
        # Retourne la hauteur du joueur (64 pixels).
        return self.height

    def getWidth(self):
        # Retourne la largeur du joueur (64 pixels).
        return self.width

    def isMoveable(self):
        # Indique si l'objet peut être déplacé (toujours vrai pour le joueur).
        return True

    def canBeCovered(self):
        # Indique si un autre objet peut être placé dessus (toujours vrai pour le joueur).
        return True

    def moveInCanvas(self, direction):
        # Cette méthode permet de déplacer le joueur sur le canevas dans la direction demandée.
        # La méthode utilise les coordonnées actuelles du joueur et la méthode `positionTowards` 
        # pour calculer la nouvelle position et déplacer l'image du joueur sur le canevas.
        if direction == Direction.Left:
            self.canvas.move(self.player_id, -64, 0)
            self.position = self.position.positionTowards(direction, 1)
        elif direction == Direction.Up:
            self.canvas.move(self.player_id, 0, -64)
            self.position = self.position.positionTowards(direction, 1)
        elif direction == Direction.Right:
            self.canvas.move(self.player_id, 64, 0)
            self.position = self.position.positionTowards(direction, 1)
        elif direction == Direction.Down:
            self.canvas.move(self.player_id, 0, 64)
            self.position = self.position.positionTowards(direction, 1)

    """
        Retourne True si le Mover peut se déplacer dans la direction demandée.
        Le calcul necessite de voir l'élément adjacent mais aussi l'élément suivant (offset de 2)
    """
    def canMove(self, direction):
        # Tester si la place suivante est une caisse et si après la caisse il y a une place vide pour aller dans cette direction
        if (self.wharehouse.at(self.position.positionTowards(direction, 1)).xsbChar() == '$' and
            self.wharehouse.at(self.position.positionTowards(direction, 2)).canBeCovered()):
            # Si on rentre dans le if, cela signifie que nous avons une caisse et une place vide après, 
            # alors il faut maintenant bouger le joueur et la caisse dans cette direction
            self.wharehouse.at(self.position.positionTowards(direction, 1)).moveTowards(direction)
            # Après avoir bougé la caisse, il faut transformer sa place précédente en Floor pour que le joueur puisse se déplacer dessus
            self.wharehouse.atPut(self.position.positionTowards(direction, 2), self.wharehouse.at(self.position.positionTowards(direction, 1)))
            self.wharehouse.atPut(self.position.positionTowards(direction, 1), Floor())
        return self.wharehouse.at(self.position.positionTowards(direction, 1)).canBeCovered()

    """
        Pour le déplacement, il faut penser à déplacer éventuellemnt le Box et ensuite déplacer le Mover
    """
    def moveTowards(self, direction):
        # Cette fonction modifie l'image du joueur en fonction de la direction et vérifie si le joueur peut se déplacer dans la direction 
        # demandée en appelant canMove, puis déplace effectivement le joueur si la condition est vraie. 
        # Sinon, elle exécutera une animation.
        self.setupImageForDirection(direction) 
        if self.canMove(direction): 
            self.moveInCanvas(direction)
        else:
            self.startImpossiblePushAnimation()

    """
        Le Mover est représenté differemment suivant la direction de déplacement
    """
    def setupImageForDirection(self, direction):
        # Cette méthode permet de modifier l'image de player (joueur) selon son mouvement 
        if direction == Direction.Left:
            self.image = tk.PhotoImage(file='playerLeft.png')
        elif direction == Direction.Up:
            self.image = tk.PhotoImage(file='playerUp.png')
        elif direction == Direction.Right:
            self.image = tk.PhotoImage(file='playerRight.png')
        elif direction == Direction.Down:
            self.image = tk.PhotoImage(file='playerDown.png')   
        self.player_id = self.canvas.create_image(
            self.position.getX() * 64,
            self.position.getY() * 64,
            image=self.image,
            anchor="nw",
            tags='movable'
        )

    def push(self, direction):
        # Cette méthode permet de déplacer un objet (par exemple une boîte) dans la direction spécifiée.
        # Elle effectue plusieurs actions en fonction de la direction et de l'état de l'objet.
        self.setupImageForDirection(direction) 
        if not self.canMove(direction):
            self.startImpossiblePushAnimation()
            return
        self.moveTowards(direction)

    def xsbChar(self):
        # Renvoie le caractère du joueur dans la matrice XSB
        if self.under.isFreePlace(): 
            return '@'
        else: 
            return '+'

    def isFreePlace(self):
        # Indique que la case actuelle n'est pas une place libre
        return False

    def startImpossiblePushAnimation(self):
        """Démarre une animation lorsque le joueur ne peut pas déplacer un objet."""
        if self.impossible_animation_running:
            return  # Empêche plusieurs animations simultanées

        self.impossible_animation_running = True
        x_center = self.position.getX() * 64 + 32
        y_center = self.position.getY() * 64 + 32

        def animate():
            for size in range(10, 50, 5):  # Les cercles grandissent progressivement
                if not self.impossible_animation_running:
                    break
                circle = self.canvas.create_oval(
                    x_center - size, y_center - size,
                    x_center + size, y_center + size,
                    outline="red",
                    width=2
                )
                self.canvas.update()  # Rafraîchit le canvas pour montrer l'animation
                time.sleep(0.05)  # Pause pour l'effet
                self.canvas.delete(circle)  # Supprime le cercle après affichage
            self.impossible_animation_running = False

        self.canvas.after(0, animate)

    def stopImpossiblePushAnimation(self):
        """Arrête l'animation en cours."""
        self.impossible_animation_running = False

    def cleanUpAnimation(self):
        pass

    def impossiblePushAnimation(self):
        pass


"""
    Le jeux avec tout ce qu'il faut pour dessiner et stocker/gérer la matrice d'éléments
    
"""
class Level(object):
    def __init__(self, root, xsbMatrix):
        # Cette méthode est le constructeur de la classe `Level`. Elle initialise le niveau du jeu,
        # crée le plan de l'entrepôt et configure le canevas pour l'interface graphique du jeu.

        self.root = root
        #  C'est le conteneur dans lequel le canevas et les autres éléments de l'interface graphique sont placés.

        self.wharehouse = WharehousePlan()
        # `wharehouse`  contient la matrice qui
        # représente  l'entrepôt (avec les murs, les cases vides, les boîtes, etc.).

        # Calcul des dimensions de la matrice à partir de xsbMatrix
        nbrows = len(xsbMatrix)  # Le nombre de lignes dans la matrice (représente la hauteur de l'entrepôt).
        nbcolumns = 0  # Le nombre de colonnes, initialisé à zéro.

        for line in xsbMatrix:
            nbc = len(line)  # Calcul du nombre de colonnes pour chaque ligne de la matrice.
            if nbc > nbcolumns:
                nbcolumns = nbc  # Mise à jour du nombre maximal de colonnes trouvé.

        self.height = nbrows * 64 
        # La hauteur du niveau est égale au nombre de lignes multiplié par la taille de la case  (64 pixels).
        
        self.width = nbcolumns * 64 
        # La largeur du niveau est égale au nombre maximal de colonnes multiplié par la taille de la case (64 pixels).

        self.canvas = tk.Canvas(self.root, width=self.width, height=self.height, bg="white")
        # Création d'un canevas Tkinter avec les dimensions calculées (hauteur et largeur), 
        # sur lequel seront dessinés les éléments de l'entrepôt.
        self.canvas.pack()  # Le canevas est ajouté à la fenêtre principale.

        self.player = None
        # L'attribut `player` sera utilisé pour stocker une référence au joueur.
        
        self.box = None
        # L'attribut `box` sera utilisé pour stocker une référence à une boîte (représentée par une instance de la classe `Box`).

        self.initWharehouseFromXsb(xsbMatrix)
        # Initialisation de l'entrepôt à partir de la matrice `xsbMatrix`.

        self.root.bind("<Key>", self.keypressed)
        # Liaison des événements clavier pour permettre au joueur de se déplacer.

    def initWharehouseFromXsb(self, xsbMatrix):
        # Légende des symboles :
        #   '#' = mur,  '$' = boîte, '.' = objectif, '*' = boîte sur objectif, '@' = joueur, 
        #   '+' = joueur sur objectif, '-' = sol, ' ' = sol.
        
        for ligne in range(len(xsbMatrix)):
            # Pour chaque ligne de la matrice xsbMatrix, nous allons initialiser les éléments de cette ligne.
            matrice_ELEMENT = []  # Cette liste contiendra les éléments correspondant à la ligne de la matrice.
            
            for colonne in range(len(xsbMatrix[ligne])):
                e = xsbMatrix[ligne][colonne]  # Récupère l'élément à la position (ligne, colonne) dans la matrice.
                
                if e == '#':  # Si l'élément est un mur
                    matrice_ELEMENT.append(Wall(self.canvas, Position(colonne, ligne)))  # Ajouter un mur à la matrice d'éléments.
                
                elif e == '@':  # Si l'élément est le joueur
                    self.player = Mover(self.canvas, self.wharehouse, Position(colonne, ligne), False)  # Créer un joueur à la position donnée.
                    matrice_ELEMENT.append(self.player)  # Ajouter le joueur à la liste des éléments.
                
                elif e == '$':  # Si l'élément est une boîte
                    self.box = Box(self.canvas, self.wharehouse, Position(colonne, ligne), False)  # Créer une boîte à la position donnée.
                    matrice_ELEMENT.append(self.box)  # Ajouter la boîte à la liste des éléments.
                
                elif e == '*':  # Si l'élément est une boîte sur un objectif
                    matrice_ELEMENT.append(Box(self.canvas, self.wharehouse, Position(colonne, ligne), True))  # Créer une boîte sur un objectif.
                
                elif e == '+':  # Si l'élément est un joueur sur un objectif
                    matrice_ELEMENT.append(Mover(self.canvas, self.wharehouse, Position(colonne, ligne), True))  # Créer un joueur sur un objectif.
                
                elif e == '.':  # Si l'élément est un objectif
                    matrice_ELEMENT.append(Goal(self.canvas, Position(colonne, ligne)))  # Créer un objectif à la position donnée.
                
                else:  # Si l'élément est un sol vide (représenté par un espace ' ')
                    matrice_ELEMENT.append(Floor())  # Créer un sol vide.
            
            # Ajouter la ligne d'éléments à la matrice de l'entrepôt.
            self.wharehouse.appendRow(matrice_ELEMENT)
        
        # Après avoir ajouté tous les éléments, s'assurer que les éléments mobiles (comme les joueurs et boîtes)
        # soient dessinés par-dessus les éléments statiques (comme les murs et les objectifs).
        self.canvas.tag_raise("movable", "static")

    def keypressed(self, event):
        # Gestion des événements clavier pour déplacer le joueur.
        if event.keysym == 'Left':
            self.player.moveTowards(Direction.Left)
        elif event.keysym == 'Right':
            self.player.moveTowards(Direction.Right)   
        elif event.keysym == 'space':
            self.player.moveTowards(Direction.Up)   
        elif event.keysym == 'Up':
            self.player.moveTowards(Direction.Up)    
        elif event.keysym == 'Down':
            self.player.moveTowards(Direction.Down)


# créer une interface utilisateur simple pour sélectionner un niveau Sokoban et démarrer le jeu avec le niveau choisi
class Sokoban(object):
    '''
    Main Level class
    '''
    def __init__(self):
        # Initialisation de la fenêtre principale
        self.root = tk.Tk()
        self.root.title("Sokoban")

        # Texte d'accueil affiché en haut de la fenêtre
        text = tk.Label(
            self.root,
            text='Bienvenue sur Sokoban',  # Message de bienvenue
            bd=100,  # Espacement autour du texte
            font=('Georgia', 48),  # Police utilisée pour le texte
            bg="white"  # Couleur de fond du texte
        )
        text.pack()

        # Création d'une variable pour stocker le niveau sélectionné
        self.level_var = tk.StringVar(self.root)

        # Définir une valeur par défaut pour la liste déroulante
        self.level_var.set("sélectionner un niveau")

        # Génération des options de niveau avec une expression "f-string"
        level_options = [f"Level {i}" for i in range(1, len(SokobanXSBLevels) + 1)]

        # Création d'une liste déroulante pour choisir un niveau
        level_combobox = tk.OptionMenu(self.root, self.level_var, *level_options)
        level_combobox.pack(pady=10)

        # Application de styles à la liste déroulante
        font_style = ("Georgia", 24, "bold")  # Style de police pour la liste
        level_combobox.configure(font=font_style, bg="yellow")  # Couleur de fond de la liste

        # Création d'un bouton pour démarrer le jeu
        start_button = tk.Button(
            self.root,
            text="Start",  # Texte affiché sur le bouton
            font=('Georgia', 24),  # Style de police utilisé pour le bouton
            command=self.startSelectedLevel,  # Méthode appelée lorsque le bouton est cliqué
            bg="green",  # Couleur de fond du bouton
            fg="white"  # Couleur du texte sur le bouton
        )
        start_button.pack(pady=10)

    def startSelectedLevel(self):
        '''
        Méthode pour démarrer un niveau choisi.
        '''
        # Récupère le niveau sélectionné à partir de la liste déroulante
        if self.level_var.get() == "sélectionner un niveau":
            selected_level = 1  # Si aucun niveau n'est choisi, on démarre au niveau 1
        else:
            # Récupère le numéro du niveau sélectionné (convertit le texte en entier)
            selected_level = int(self.level_var.get().split()[1])

        # Vérifie si le niveau sélectionné est valide
        if 1 <= selected_level <= len(SokobanXSBLevels):
            # Ferme la fenêtre principale
            self.root.destroy()

            # Création d'une nouvelle fenêtre pour le niveau de jeu
            game_window = tk.Tk()
            game_window.title("Sokoban")  # Définit le titre de la nouvelle fenêtre

            # Initialise et affiche le niveau sélectionné
            level = Level(game_window, SokobanXSBLevels[selected_level])

            # Lance la boucle principale pour afficher le niveau
            game_window.mainloop()

    def play(self):
        '''
        Méthode pour démarrer l'application et afficher le menu principal.
        '''
        self.root.mainloop()


# Lancer le jeu pour afficher le menu principal et choisir un niveau
Sokoban().play()
